In [1]:
%pwd

'd:\\Projects\\chicken-disease-classification\\research'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


In [9]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [12]:
class ConfiguratonManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            ):
        print("Configuration Manager Initiated")
        print(f"Config File Path: {config_filepath}, with dtype {type(config_filepath)}")
        print(f"Params File Path: {params_filepath} with dtype {type(params_filepath)}")
        self.configs = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        print(f"Configs: {self.configs}")
        create_directories([self.configs.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        return DataIngestionConfig(
            root_dir = Path(config.root_dir),
            source_URL = config.source_URL,
            local_data_file = Path(config.local_data_file),
            unzip_dir = Path(config.unzip_dir)
        )

In [5]:
import os
import urllib.request as request
from zipfile import ZipFile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [6]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"Downloaded file with the folowing headers: \n{headers}")
        
        else:
            logger.info(f"File already exists at {self.config.local_data_file} of size {get_size(self.config.local_data_file)}")

        def extract_zip_file(self):

            unzip_path = self.config.unzip_dir
            os.makedirs(unzip_path, exist_ok=True)
            with ZipFile(self.config.local_data_file, 'r') as zip:
                zip.extractall(path=unzip_path)
                logger.info(f"Extracted file at {self.config.unzip_dir}")
                

In [14]:
try:
    config = ConfiguratonManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    logger.exception(e)
    raise e
    

Configuration Manager Initiated
Config File Path: D:\Projects\chicken-disease-classification\config\config.yaml, with dtype <class 'pathlib.WindowsPath'>
Params File Path: D:\Projects\chicken-disease-classification\params.yaml with dtype <class 'pathlib.WindowsPath'>
Inside read_yaml func before try clause: D:\Projects\chicken-disease-classification\config\config.yaml,and type: <class 'pathlib.WindowsPath'>
True
Inside read_yaml function's try: D:\Projects\chicken-disease-classification\config\config.yaml, and type: <class 'pathlib.WindowsPath'>
[ 2023-09-14 00:16:50,452: INFO: common:  yaml file: D:\Projects\chicken-disease-classification\config\config.yaml loaded successfully]
Inside read_yaml func before try clause: D:\Projects\chicken-disease-classification\params.yaml,and type: <class 'pathlib.WindowsPath'>
True
Inside read_yaml function's try: D:\Projects\chicken-disease-classification\params.yaml, and type: <class 'pathlib.WindowsPath'>
[ 2023-09-14 00:16:50,459: INFO: common:  

BoxKeyError: "'ConfigBox' object has no attribute 'artifacts_root'"